In [1]:
import os
import warnings

# Configuraciones para reducir warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("✓ TOKENIZERS_PARALLELISM configurado a 'false' para evitar warnings")

# Filtros específicos para warnings conocidos
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", message=".*bert.pooler.*")
warnings.filterwarnings("ignore", message=".*Asking to truncate.*")
warnings.filterwarnings("ignore", message=".*Some weights of the model.*")

print("✓ Configuración de warnings completada - BERT warnings suprimidos")

✓ TOKENIZERS_PARALLELISM configurado a 'false' para evitar warnings
✓ Configuración de warnings completada - BERT warnings suprimidos


# Modelado de Tópicos con FASTopic

Este notebook implementa un análisis de tópicos usando FASTopic sobre las opiniones turísticas clasificadas. Utiliza LangChain con GPT-4o-mini para asignar nombres semánticamente coherentes a los tópicos identificados y compara resultados con BERTopic.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import sys
from dotenv import load_dotenv

sys.path.append('../scripts')

from topicos import (
    configurar_clasificador_topicos, 
    configurar_fastopic_inteligente,
    LimpiadorTextoMejorado, 
    mostrar_ejemplos_limpieza,
    evaluar_modelo_topicos,
    extraer_palabras_fastopic,
    mostrar_metricas
)

from topicos.utils_topicos import (
    procesar_topicos_fastopic,
    obtener_asignaciones_topicos_fastopic,
    visualizar_distribucion_topicos_fastopic,
    mostrar_ejemplos_por_topico_fastopic,
    generar_reporte_fastopic
)

load_dotenv()

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

Device set to use cuda:0
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/victorwkey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/victorwkey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
clasificador_topicos = configurar_clasificador_topicos()

In [4]:
CIUDAD_ANALIZAR = "Mazatlan"
df = pd.read_csv('../data/processed/dataset_opiniones_analisis.csv')

if 'TopicoConFASTopic' not in df.columns:
    df['TopicoConFASTopic'] = np.nan

print(f"Dataset cargado: {df.shape[0]} opiniones")
print(f"Distribución total por ciudad:")
print(df['Ciudad'].value_counts())

df_ciudad = df[df['Ciudad'] == CIUDAD_ANALIZAR].copy()

columna_texto = 'TituloReviewLimpio'

texts = df_ciudad[columna_texto].dropna().tolist()

Dataset cargado: 2457 opiniones
Distribución total por ciudad:
Ciudad
Mazatlan           499
Puebla             497
Puerto_vallarta    489
Cdmx               489
Cancun             483
Name: count, dtype: int64


In [5]:
columna_limpia = "TituloReviewLimpio"

limpiar_de_nuevo = False

if columna_limpia not in df.columns or limpiar_de_nuevo:
    
    limpiador = LimpiadorTextoMejorado(idiomas=['spanish', 'english'])
    
    print(f"🔍 Iniciando limpieza para {CIUDAD_ANALIZAR}...")
    
    df = limpiador.limpiar_dataframe(
        df,
        columna_texto='TituloReview',
        nombre_columna_limpia=columna_limpia,
        aplicar_traduccion=True,
        filtrar_adjetivos=True,  # Nueva opción para filtrar adjetivos
        filtrar_solo_espanol=True,  # Nueva opción para filtrar solo textos en español
        aplicar_lematizacion=True,
        min_longitud_palabra=2,
        max_palabras=None,
        mostrar_estadisticas=True  # Mostrar estadísticas detalladas
    )
    
    print(f"\n💾 Guardando dataset procesado...")
    df.to_csv('../data/processed/dataset_opiniones_analisis.csv', index=False)
    print(f"✅ Dataset guardado exitosamente")

df_ciudad = df[df['Ciudad'] == CIUDAD_ANALIZAR].copy()
texts = df_ciudad[columna_texto].dropna().tolist()

print(f"\n🎯 RESUMEN FINAL PARA {CIUDAD_ANALIZAR}:")
print(f"   📝 Textos disponibles: {len(texts):,}")
if texts:
    promedio = sum(len(t.split()) for t in texts) / len(texts)
    print(f"   📊 Promedio palabras: {promedio:.1f}")
    print(f"   📏 Rango: {min(len(t.split()) for t in texts)} - {max(len(t.split()) for t in texts)} palabras")
    print(f"   🎲 Muestra: '{texts[0][:50]}...'" if texts else "")


🎯 RESUMEN FINAL PARA Mazatlan:
   📝 Textos disponibles: 499
   📊 Promedio palabras: 56.3
   📏 Rango: 12 - 476 palabras
   🎲 Muestra: 'divertido y estar mucho con mazatlar mx el gente a...'


In [6]:
model, top_words, doc_topic_dist, reporte_optimizacion, tiempo_entrenamiento = configurar_fastopic_inteligente(texts)
    
print("🤖 Configuración automática de FASTopic completada")
print(reporte_optimizacion)

parsing texts: 100%|██████████| 499/499 [00:08<00:00, 62.24it/s]
2025-09-24 13:52:57,927 - TopMost - Real vocab size: 549
2025-09-24 13:52:57,929 - TopMost - Real training size: 499 	 avg length: 16.126
parsing texts: 100%|██████████| 499/499 [00:08<00:00, 62.24it/s]
2025-09-24 13:52:57,927 - TopMost - Real vocab size: 549
2025-09-24 13:52:57,929 - TopMost - Real training size: 499 	 avg length: 16.126
Training FASTopic: 100%|██████████| 200/200 [00:18<00:00, 10.82it/s]

parsing texts: 100%|██████████| 499/499 [00:05<00:00, 84.57it/s] 
2025-09-24 13:53:32,585 - TopMost - Real vocab size: 549
2025-09-24 13:53:32,586 - TopMost - Real training size: 499 	 avg length: 16.128
parsing texts: 100%|██████████| 499/499 [00:05<00:00, 84.57it/s]
2025-09-24 13:53:32,585 - TopMost - Real vocab size: 549
2025-09-24 13:53:32,586 - TopMost - Real training size: 499 	 avg length: 16.128
Training FASTopic: 100%|██████████| 200/200 [00:25<00:00,  7.73it/s]

parsing texts: 100%|██████████| 499/499 [00:08<

🤖 Configuración automática de FASTopic completada

📊 CONFIGURACIÓN Y OPTIMIZACIÓN AUTOMÁTICA DE FASTOPIC

📈 Análisis del Corpus:
  📄 Documentos: 499
  📝 Palabras promedio: 56.3
  🔤 Vocabulario único: 3,154

🎯 Optimización de Tópicos:
  🔍 Modelos evaluados: 6
  🏷️ K óptimo encontrado: 5
  📈 Coherencia CV: 0.4852
  🔄 Diversidad: 1.0000
  ✅ Criterio diversidad (≥0.98): Cumplido

🔧 Configuración Final:
  🌍 Modelo embeddings: paraphrase-multilingual-MiniLM-L12-v2
  🔧 Tokenizer: Multiidioma (ES, EN, PT, FR, IT)
  🚀 Épocas entrenamiento: 200 (para todos los candidatos)
  ⏱️ Tiempo total optimización: 248.48s



In [7]:
# La optimización y entrenamiento ya se realizó en la celda anterior
topic_model = model
print("✅ Modelo FASTopic optimizado y entrenado automáticamente")

✅ Modelo FASTopic optimizado y entrenado automáticamente


In [8]:
import numpy as np

def select_topics_stat(doc_vec, threshold, max_topics=3):
    """
    Selecciona tópicos estadísticamente mejores que azar (95% confianza).
    - Usa un threshold ya calculado.
    - Máximo max_topics tópicos seleccionados.
    """
    doc_vec = np.array(doc_vec)
    order = np.argsort(doc_vec)[::-1]

    selected = []
    for i in order:
        if doc_vec[i] > threshold:
            selected.append(i)
        if len(selected) >= max_topics:
            break

    return selected

# Calcular solo una vez
N = len(doc_topic_dist)
C = doc_topic_dist.shape[1]
p = 1 / C
sigma = np.sqrt(p * (1 - p) / N)
threshold = p + 1.96 * sigma

# Aplicar a todos los documentos
for i, doc in enumerate(doc_topic_dist, 1):
    selected = select_topics_stat(doc, threshold)
    probs_str = " | ".join(f"{p:.2f}" for p in doc)
    print(f"Documento {i}: [{probs_str}] -> Tópicos seleccionados: {selected}, Umbral: {threshold:.3f}")


Documento 1: [0.14 | 0.09 | 0.53 | 0.10 | 0.14] -> Tópicos seleccionados: [2], Umbral: 0.235
Documento 2: [0.11 | 0.13 | 0.09 | 0.07 | 0.59] -> Tópicos seleccionados: [4], Umbral: 0.235
Documento 3: [0.15 | 0.15 | 0.44 | 0.07 | 0.19] -> Tópicos seleccionados: [2], Umbral: 0.235
Documento 4: [0.02 | 0.61 | 0.10 | 0.03 | 0.24] -> Tópicos seleccionados: [1, 4], Umbral: 0.235
Documento 5: [0.14 | 0.10 | 0.07 | 0.11 | 0.58] -> Tópicos seleccionados: [4], Umbral: 0.235
Documento 6: [0.03 | 0.26 | 0.48 | 0.02 | 0.21] -> Tópicos seleccionados: [2, 1], Umbral: 0.235
Documento 7: [0.09 | 0.02 | 0.14 | 0.72 | 0.02] -> Tópicos seleccionados: [3], Umbral: 0.235
Documento 8: [0.07 | 0.06 | 0.79 | 0.02 | 0.06] -> Tópicos seleccionados: [2], Umbral: 0.235
Documento 9: [0.05 | 0.23 | 0.53 | 0.03 | 0.16] -> Tópicos seleccionados: [2], Umbral: 0.235
Documento 10: [0.08 | 0.10 | 0.19 | 0.58 | 0.05] -> Tópicos seleccionados: [3], Umbral: 0.235
Documento 11: [0.06 | 0.14 | 0.52 | 0.11 | 0.16] -> Tópicos sel

In [9]:
topic_info = procesar_topicos_fastopic(topic_model, doc_topic_dist, top_words)

# Preparar información de todos los tópicos para procesamiento en lote
topics_info_text = ""
for _, row in topic_info.iterrows():
    topic_id = row['Topic']
    keywords = row['Keywords']
    if keywords and keywords not in ["Sin palabras", "Error en procesamiento"]:
        topics_info_text += f"Tópico {topic_id}: {keywords}\n"

resultado = clasificador_topicos.invoke({"topics_info": topics_info_text})

topic_names = {}
for topic_label in resultado.topics:
    topic_names[topic_label.topic_id] = topic_label.label

topic_info['Name'] = topic_info['Topic'].map(topic_names)

In [10]:
print(topics_info_text)

Tópico 0: isla, acuario, barco, espectaculo, persona, bote, bebida, hora, viaje, almuerzo
Tópico 1: cristal, faro, caminata, puente, vistas, subir, mirador, escalón, ejercicio, vista
Tópico 2: musico, noche, ambiente, plaza, vendedor, area, banda, iluminacion, iluminado, teatro
Tópico 3: playa, ola, nadar, mar, bruja, oleaje, hotel, cerrito, cerca, basura
Tópico 4: catedral, arquitectura, historia, edificio, iglesia, viejo, centro, ciudad, antiguo, interior



In [11]:
print(resultado)

topics=[TopicLabel(topic_id=0, label='Excursiones Marítimas'), TopicLabel(topic_id=1, label='Miradores'), TopicLabel(topic_id=2, label='Vida Nocturna'), TopicLabel(topic_id=3, label='Playas'), TopicLabel(topic_id=4, label='Patrimonio Cultural')]


In [12]:
# Obtener asignaciones de tópicos para documentos
topic_assignments, topic_names_assigned, topic_probabilities = obtener_asignaciones_topicos_fastopic(
    doc_topic_dist, 
    topic_names, 
    threshold=0.1
)

# Procesar solo datos de la ciudad seleccionada
df_con_topicos = df_ciudad.dropna(subset=[columna_texto]).copy()
df_con_topicos['Topico'] = topic_assignments
df_con_topicos['Topico_Nombre'] = topic_names_assigned
df_con_topicos['Probabilidad_Topico'] = topic_probabilities

# Verificar si la ciudad ya tenía resultados previos
indices_ciudad = df_con_topicos.index
opiniones_previas = df.loc[indices_ciudad, 'TopicoConFASTopic'].notna().sum()

if opiniones_previas > 0:
    print(f"🔄 SOBRESCRIBIENDO resultados previos para {CIUDAD_ANALIZAR}:")
    print(f"   📝 Opiniones con tópicos previos: {opiniones_previas}")
    print(f"   🆕 Nuevas asignaciones de tópicos: {len(df_con_topicos)}")
else:
    print(f"🆕 PRIMERA VEZ procesando {CIUDAD_ANALIZAR}:")
    print(f"   📝 Nuevas asignaciones de tópicos: {len(df_con_topicos)}")

# Actualizar el dataset original con los tópicos de la ciudad analizada
df.loc[indices_ciudad, 'TopicoConFASTopic'] = df_con_topicos['Topico_Nombre']

print(f"\n✅ Actualización completada para {CIUDAD_ANALIZAR}")
print(f"Distribución de tópicos en {CIUDAD_ANALIZAR}:")
topico_counts = df_con_topicos['Topico_Nombre'].value_counts()
print(topico_counts)

print(f"\nPorcentaje de opiniones por tópico en {CIUDAD_ANALIZAR}:")
topico_pct = (topico_counts / len(df_con_topicos) * 100).round(2)
for topico, pct in topico_pct.items():
    print(f"{topico}: {pct}%")

print(f"\nEstado actualización dataset completo:")
print(f"Total opiniones: {len(df)}")
print(f"Opiniones con tópico FASTopic asignado: {df['TopicoConFASTopic'].notna().sum()}")
print(f"Opiniones pendientes: {df['TopicoConFASTopic'].isna().sum()}")

🆕 PRIMERA VEZ procesando Mazatlan:
   📝 Nuevas asignaciones de tópicos: 499

✅ Actualización completada para Mazatlan
Distribución de tópicos en Mazatlan:
Topico_Nombre
Playas                   129
Patrimonio Cultural      103
Vida Nocturna            100
Excursiones Marítimas     98
Miradores                 69
Name: count, dtype: int64

Porcentaje de opiniones por tópico en Mazatlan:
Playas: 25.85%
Patrimonio Cultural: 20.64%
Vida Nocturna: 20.04%
Excursiones Marítimas: 19.64%
Miradores: 13.83%

Estado actualización dataset completo:
Total opiniones: 2457
Opiniones con tópico FASTopic asignado: 1471
Opiniones pendientes: 986


In [13]:
# Obtener asignaciones de tópicos para documentos
topic_assignments, topic_names_assigned, topic_probabilities = obtener_asignaciones_topicos_fastopic(
    doc_topic_dist, 
    topic_names, 
    threshold=0.1
)

# Procesar solo datos de la ciudad seleccionada
df_con_topicos = df_ciudad.dropna(subset=[columna_texto]).copy()
df_con_topicos['Topico'] = topic_assignments
df_con_topicos['Topico_Nombre'] = topic_names_assigned
df_con_topicos['Probabilidad_Topico'] = topic_probabilities

# Actualizar el dataset original con los tópicos de la ciudad analizada
indices_ciudad = df_con_topicos.index
df.loc[indices_ciudad, 'TopicoConFASTopic'] = df_con_topicos['Topico_Nombre']

In [14]:
# Mostrar ejemplos de opiniones por tópico
mostrar_ejemplos_por_topico_fastopic(
    df_con_topicos, 
    topico_col='Topico_Nombre',
    texto_col='TituloReview',
    n_ejemplos=3,
    top_n_topicos=5
)

📚 EJEMPLOS DE OPINIONES POR TÓPICO

🏷️ 1. Playas
📊 Total de opiniones: 129
📝 Ejemplos:
   1. Espectacular. Todo me Encanto. Las comidas el acuario, Las compa en el mercado el ambiente en la pla...
   2. Paseo en familia. Un paseo excelente! La playa un poco fuerte el oleaje pero muy limpia, las bandas ...
   3. Largo y silencioso. 12 millas de excelentes vistas de la playa y el océano, y algunos bares en la pl...
----------------------------------------------------------------------

🏷️ 2. Patrimonio Cultural
📊 Total de opiniones: 103
📝 Ejemplos:
   1. Un mes en Mazatlán. Acabamos de pasar un mes en Mazatlán. No estoy de acuerdo con algunas críticas. ...
   2. Nunca más, por desgracia. Mazatlán no es seguro. Lamento escribir esto, pero estuvimos allí una sema...
   3. Mazatlán es hermoso. Denunciamos un condominio increíble justo en el Malecón. Ventanas de piso a tec...
----------------------------------------------------------------------

🏷️ 3. Vida Nocturna
📊 Total de opiniones: 100

In [15]:
# ===== COMPARACIÓN TEXTO ORIGINAL VS TEXTO LIMPIO =====
if 'TituloReviewLimpio' in df_con_topicos.columns:
    print(f"🔍 Comparación de ejemplos - Original vs Limpio ({CIUDAD_ANALIZAR}):")
    print("=" * 80)
    
    # Seleccionar algunos ejemplos para mostrar la diferencia
    ejemplos_muestra = df_con_topicos.sample(n=min(10, len(df_con_topicos)))
    
    for i, (_, row) in enumerate(ejemplos_muestra.iterrows(), 1):
        print(f"\n📄 EJEMPLO {i} - Tópico: {row['Topico_Nombre']}")
        print(f"🔸 Original: {row['TituloReview']}")
        print(f"🔹 Limpio:   {row['TituloReviewLimpio']}")
        print(f"📊 Probabilidad: {row['Probabilidad_Topico']:.3f}")
        
        # Calcular estadísticas del ejemplo
        len_orig = len(str(row['TituloReview']))
        len_limpio = len(str(row['TituloReviewLimpio']))
        reduccion = ((len_orig-len_limpio)/len_orig*100) if len_orig > 0 else 0
        print(f"📉 Reducción: {len_orig} → {len_limpio} caracteres ({reduccion:.1f}%)")
        print("-" * 80)

🔍 Comparación de ejemplos - Original vs Limpio (Mazatlan):

📄 EJEMPLO 1 - Tópico: Patrimonio Cultural
🔸 Original: espectacular remodelación. El centro histórico de viejo Mazatlán es una de los lugares donde tienes que venir cuando piensas en Mazatlán piezas en playa pero el centro histórico vale la pena.
🔹 Limpio:   remodelacion el centro de mazatlan ser uno de el lugar donde tener que venir cuando pensar en mazatlar pieza en playa pero el centro historico valer el pena
📊 Probabilidad: 0.448
📉 Reducción: 191 → 156 caracteres (18.3%)
--------------------------------------------------------------------------------

📄 EJEMPLO 2 - Tópico: Playas
🔸 Original: Hemosa playa. Una playa que se encuentra alejada del bullicio de la ciudad en donde puedes encontrar paz y puedes realizar caminatas extensas disfrutando su hermoso paisaje.
🔹 Limpio:   hemós playa uno playa que él encontrar del bullicio de el ciudad en donde poder encontrar paz y poder realizar caminata disfrutar su paisaje
📊 Probabili

In [16]:
# ===== VISUALIZACIONES ESPECÍFICAS DE FASTOPIC =====

# Generar visualización de tópicos
fig_topics = topic_model.visualize_topic(top_n=min(8, len(topic_info)))
fig_topics.update_layout(title=f"Palabras Principales por Tópico - {CIUDAD_ANALIZAR}")
fig_topics.show()

# Generar visualización de pesos de tópicos
fig_weights = topic_model.visualize_topic_weights(top_n=min(10, len(topic_info)))
fig_weights.update_layout(title=f"Distribución de Pesos de Tópicos - {CIUDAD_ANALIZAR}")
fig_weights.show()

# Generar jerarquía de tópicos si hay suficientes
if len(topic_info) >= 3:
    fig_hierarchy = topic_model.visualize_topic_hierarchy()
    fig_hierarchy.update_layout(title=f"Jerarquía de Tópicos - {CIUDAD_ANALIZAR}")
    fig_hierarchy.show()

print(f"\n📋 Resumen del modelado FASTopic:")
print(f"✅ Textos analizados: {len(texts)}")
print(f"🎯 Tópicos encontrados: {len(topic_info)}")
print(f"⏱️ Tiempo de entrenamiento: {tiempo_entrenamiento:.2f}s")
print(f"📊 Probabilidad promedio: {df_con_topicos['Probabilidad_Topico'].mean():.3f}")
print(f"📈 Distribución:")
for i, (topico, count) in enumerate(topico_counts.head(5).items()):
    print(f"   {i+1}. {topico}: {count} opiniones ({count/len(df_con_topicos)*100:.1f}%)")


📋 Resumen del modelado FASTopic:
✅ Textos analizados: 499
🎯 Tópicos encontrados: 5
⏱️ Tiempo de entrenamiento: 248.48s
📊 Probabilidad promedio: 0.623
📈 Distribución:
   1. Playas: 129 opiniones (25.9%)
   2. Patrimonio Cultural: 103 opiniones (20.6%)
   3. Vida Nocturna: 100 opiniones (20.0%)
   4. Excursiones Marítimas: 98 opiniones (19.6%)
   5. Miradores: 69 opiniones (13.8%)


In [17]:
# ===== COMPARACIÓN CON BERTOPIC =====

if 'TopicoConBERTopic' in df.columns:
    print(f"🔄 COMPARACIÓN FASTopic vs BERTopic en {CIUDAD_ANALIZAR}")
    print("=" * 60)
    
    # Usar datos actuales procesados en lugar de df_ciudad que puede tener datos previos
    df_comparacion = df_con_topicos[
        df_con_topicos.index.isin(df_ciudad[df_ciudad['TopicoConBERTopic'].notna()].index)
    ].copy()
    
    # Agregar datos de BERTopic al dataframe de comparación
    bertopic_data = df_ciudad[df_ciudad['TopicoConBERTopic'].notna()]['TopicoConBERTopic']
    df_comparacion = df_comparacion[df_comparacion.index.isin(bertopic_data.index)].copy()
    df_comparacion['TopicoConBERTopic'] = bertopic_data
    
    if len(df_comparacion) > 0:
        print(f"📊 Opiniones con ambos análisis: {len(df_comparacion)}")
        
        # Comparar número de tópicos únicos usando datos actuales
        topicos_bertopic = df_comparacion['TopicoConBERTopic'].nunique()
        topicos_fastopic = df_comparacion['Topico_Nombre'].nunique()  # Usar datos actuales
        
        print(f"\n🎯 NÚMERO DE TÓPICOS:")
        print(f"   BERTopic: {topicos_bertopic}")
        print(f"   FASTopic: {topicos_fastopic}")
        
        # Mostrar distribuciones lado a lado
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        
        # BERTopic
        bertopic_counts = df_comparacion['TopicoConBERTopic'].value_counts().head(10)
        bertopic_counts.plot(kind='bar', ax=axes[0], color='skyblue')
        axes[0].set_title(f'Distribución BERTopic - {CIUDAD_ANALIZAR}', fontweight='bold')
        axes[0].set_xlabel('Tópicos')
        axes[0].set_ylabel('Número de Opiniones')
        axes[0].tick_params(axis='x', rotation=45)
        
        # FASTopic - usar datos actuales
        fastopic_counts = df_comparacion['Topico_Nombre'].value_counts().head(10)
        fastopic_counts.plot(kind='bar', ax=axes[1], color='lightcoral')
        axes[1].set_title(f'Distribución FASTopic - {CIUDAD_ANALIZAR}', fontweight='bold')
        axes[1].set_xlabel('Tópicos')
        axes[1].set_ylabel('Número de Opiniones')
        axes[1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Análisis de concordancia (tópicos más frecuentes)
        print(f"\n📈 TOP 5 TÓPICOS POR MÉTODO:")
        print(f"\n🔵 BERTopic:")
        for i, (topico, count) in enumerate(bertopic_counts.head(5).items(), 1):
            pct = (count / len(df_comparacion) * 100)
            print(f"   {i}. {topico}: {count} ({pct:.1f}%)")
        
        print(f"\n🔴 FASTopic:")
        for i, (topico, count) in enumerate(fastopic_counts.head(5).items(), 1):
            pct = (count / len(df_comparacion) * 100)
            print(f"   {i}. {topico}: {count} ({pct:.1f}%)")
            
        print(f"\n💡 OBSERVACIONES:")
        print(f"   • FASTopic identificó {topicos_fastopic} tópicos vs {topicos_bertopic} de BERTopic")
        if topicos_fastopic > topicos_bertopic:
            print(f"   • FASTopic muestra mayor granularidad en la segmentación")
        elif topicos_fastopic < topicos_bertopic:
            print(f"   • FASTopic agrupa conceptos de manera más generalizada")
        else:
            print(f"   • Ambos métodos identificaron el mismo número de tópicos")
            
        print(f"   • Tiempo FASTopic: {tiempo_entrenamiento:.2f}s (más rápido que BERTopic)")
        
        # Mostrar resumen de datos actuales de FASTopic
        print(f"\n📊 RESUMEN ACTUAL FASTopic:")
        print(f"   Total tópicos identificados: {len(topic_info)}")
        print(f"   Tópicos en datos actuales: {df_con_topicos['Topico_Nombre'].nunique()}")
        print(f"   Documentos procesados: {len(df_con_topicos)}")
        
    else:
        print(f"⚠️ No hay opiniones de {CIUDAD_ANALIZAR} con ambos análisis completados")
        print(f"   BERTopic disponible: {df_ciudad['TopicoConBERTopic'].notna().sum()} opiniones")
        print(f"   FASTopic actual: {len(df_con_topicos)} opiniones")
else:
    print(f"ℹ️ No hay análisis previo de BERTopic para comparar")
    print(f"   Ejecuta el notebook 07-modelado-de-topicos-con-bertopic.ipynb primero")
    

🔄 COMPARACIÓN FASTopic vs BERTopic en Mazatlan
⚠️ No hay opiniones de Mazatlan con ambos análisis completados
   BERTopic disponible: 0 opiniones
   FASTopic actual: 499 opiniones


In [18]:
generar_reporte_fastopic(
    topic_info, 
    len(df_con_topicos), 
    CIUDAD_ANALIZAR, 
    tiempo_entrenamiento
)

df.to_csv('../data/processed/dataset_opiniones_analisis.csv', index=False)

📋 REPORTE COMPLETO - ANÁLISIS CON FASTOPIC
🎯 ANÁLISIS COMPLETADO:
   📍 Ciudad: Mazatlan
   📄 Documentos analizados: 499
   🏷️ Tópicos identificados: 5
   ⏱️ Tiempo de entrenamiento: 248.48 segundos

📊 ESTADÍSTICAS DE TÓPICOS:
   🎯 Tópico más relevante: Playas
   📈 Peso máximo: 115.444
   📉 Peso mínimo: 77.037
   📊 Peso promedio: 99.800

🏆 TOP 5 TÓPICOS POR RELEVANCIA:
   1. Playas
      📊 Peso: 115.444 | 📄 Documentos: 224
      🔑 Palabras clave: playa, ola, nadar, mar, bruja, oleaje, hotel, cerr...
   2. Vida Nocturna
      📊 Peso: 111.168 | 📄 Documentos: 273
      🔑 Palabras clave: musico, noche, ambiente, plaza, vendedor, area, ba...
   3. Patrimonio Cultural
      📊 Peso: 108.804 | 📄 Documentos: 271
      🔑 Palabras clave: catedral, arquitectura, historia, edificio, iglesi...
   4. Excursiones Marítimas
      📊 Peso: 86.548 | 📄 Documentos: 234
      🔑 Palabras clave: isla, acuario, barco, espectaculo, persona, bote, ...
   5. Miradores
      📊 Peso: 77.037 | 📄 Documentos: 219
      

In [19]:
# ===== EVALUACIÓN DE MÉTRICAS FASTOPIC =====

print(f"🔬 Evaluando modelo FASTopic para {CIUDAD_ANALIZAR}...")

topics_words_fastopic = extraer_palabras_fastopic(top_words, words_per_topic=10)
metricas_fastopic = evaluar_modelo_topicos(texts, topics_words_fastopic, f"FASTopic - {CIUDAD_ANALIZAR}")

mostrar_metricas(metricas_fastopic)

🔬 Evaluando modelo FASTopic para Mazatlan...


📊 Evaluación FASTopic - Mazatlan:
   🎯 Tópicos: 5
   📈 Coherencia CV: 0.4852
   🔄 Diversidad: 1.0000
